In [ ]:
import os
import imageio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from glob import glob
from skimage import color

In [ ]:
sns.set_style('darkgrid')
sns.set_context('poster')

In [ ]:
filenames = sorted(glob(os.path.join('/Users', 'mnotter', 'screentime_log', 'day_*.png')))
N= 120
f = filenames[N]

In [ ]:
img = imageio.imread(f)
img = img[:340, ...]

In [ ]:
plt.figure(figsize=(15, 4))
plt.imshow(img)

In [ ]:
def find_markers(img):
    return sorted(np.argsort(np.std(color.rgb2gray(img[..., :3]), axis=0)[:975])[::-1][:2])

In [ ]:
def find_entries(img):
    return np.squeeze(np.argwhere(np.prod([img[..., 0]==31, img[..., 1]==119, img[..., 2]==180], axis=0).sum(axis=0) > 0))

In [ ]:
def get_time_points(img):
    img = img[:300, ...]
    marker = find_markers(img)
    return np.unique(np.round((find_entries(img) - marker[0]) / np.diff(marker) * 8 * 60, 0).astype('int'))

In [ ]:
marker = find_markers(img)
marker

In [ ]:
time_points = [get_time_points(imageio.imread(f)) for f in tqdm(filenames)]

In [ ]:
data = []
for t in tqdm(time_points):
    df = pd.DataFrame(t, columns=['pixel']).set_index('pixel')
    df['active'] = 1
    data.append(df)
    
# Create dataframe
data = pd.concat(data, axis=1).fillna(0)

# Drop days below 4h
data = data.iloc[:, (data.sum()>(4.2*60)).values]

# Drop false recordings (full day almost 100% active)
data = data.iloc[:, (data.sum()<13*60).values]

# Restrict time window
data = data.loc[-120:1060]

In [ ]:
# Set time as index
data['time'] = pd.to_datetime(data.index*100*10000000*60) + pd.Timedelta('09:00:00')
data = data.set_index('time').resample('180S').mean()
data['time'] = data.index.strftime('%H:%M')
data.set_index('time', inplace=True)

In [ ]:
sort_values = False
if sort_values:
    idx = np.argsort(np.argmax(data.values, axis=0))
    data = data.iloc[:, idx]

In [ ]:
day_mean = data.mean(axis=1) * 100

In [ ]:
plt.figure(figsize=(15, 20))
for i, n in enumerate(range(data.shape[1])):
    plt.plot(data.values[:, n]*i, marker='.', lw=0, markersize=2)
plt.ylim(0.5, n+0.5)
xticklabels = day_mean.index.values[::60]
xtickpos = np.argwhere([d in xticklabels for d in day_mean.index]).squeeze()
plt.xticks(ticks=xtickpos, labels=xticklabels, rotation=90)
plt.ylabel('Day')
plt.xlabel('Time')
plt.title('Detailed overview over the last %d workdays' % data.shape[1]);
plt.show();

In [ ]:
plt.figure(figsize=(16, 5))
day_mean.plot()
xticklabels = day_mean.index.values[::60]
xtickpos = np.argwhere([d in xticklabels for d in day_mean.index]).squeeze()
plt.xticks(ticks=xtickpos, labels=xticklabels, rotation=90)
plt.ylim(0, 100);
plt.ylabel('Presence [%]')
plt.title('Average Workdays [N=%d]' % data.shape[1]);